# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f14bd863ee0>
├── 'a' --> tensor([[ 1.1330, -1.7494, -0.0496],
│                   [-1.0600, -0.1308,  0.3478]])
└── 'x' --> <FastTreeValue 0x7f14bd863e20>
    └── 'c' --> tensor([[ 0.5837,  0.9151,  0.2042,  1.4552],
                        [ 0.3049, -0.9904,  1.6513, -0.3715],
                        [ 1.5892,  0.4484, -1.1044,  0.1686]])

In [4]:
t.a

tensor([[ 1.1330, -1.7494, -0.0496],
        [-1.0600, -0.1308,  0.3478]])

In [5]:
%timeit t.a

59.8 ns ± 0.0445 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f14bd863ee0>
├── 'a' --> tensor([[-1.6558, -1.1380,  0.7775],
│                   [-0.1031,  0.7079, -1.4526]])
└── 'x' --> <FastTreeValue 0x7f14bd863e20>
    └── 'c' --> tensor([[ 0.5837,  0.9151,  0.2042,  1.4552],
                        [ 0.3049, -0.9904,  1.6513, -0.3715],
                        [ 1.5892,  0.4484, -1.1044,  0.1686]])

In [7]:
%timeit t.a = new_value

58.1 ns ± 0.0408 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 1.1330, -1.7494, -0.0496],
               [-1.0600, -0.1308,  0.3478]]),
    x: Batch(
           c: tensor([[ 0.5837,  0.9151,  0.2042,  1.4552],
                      [ 0.3049, -0.9904,  1.6513, -0.3715],
                      [ 1.5892,  0.4484, -1.1044,  0.1686]]),
       ),
)

In [10]:
b.a

tensor([[ 1.1330, -1.7494, -0.0496],
        [-1.0600, -0.1308,  0.3478]])

In [11]:
%timeit b.a

54.1 ns ± 0.0504 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-1.6344,  1.1913, -0.3414],
               [-0.5470, -0.0174,  0.0759]]),
    x: Batch(
           c: tensor([[ 0.5837,  0.9151,  0.2042,  1.4552],
                      [ 0.3049, -0.9904,  1.6513, -0.3715],
                      [ 1.5892,  0.4484, -1.1044,  0.1686]]),
       ),
)

In [13]:
%timeit b.a = new_value

482 ns ± 0.16 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

855 ns ± 0.212 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.6 µs ± 52.1 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

141 µs ± 562 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

141 µs ± 238 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f13e59a8f70>
├── 'a' --> tensor([[[ 1.1330, -1.7494, -0.0496],
│                    [-1.0600, -0.1308,  0.3478]],
│           
│                   [[ 1.1330, -1.7494, -0.0496],
│                    [-1.0600, -0.1308,  0.3478]],
│           
│                   [[ 1.1330, -1.7494, -0.0496],
│                    [-1.0600, -0.1308,  0.3478]],
│           
│                   [[ 1.1330, -1.7494, -0.0496],
│                    [-1.0600, -0.1308,  0.3478]],
│           
│                   [[ 1.1330, -1.7494, -0.0496],
│                    [-1.0600, -0.1308,  0.3478]],
│           
│                   [[ 1.1330, -1.7494, -0.0496],
│                    [-1.0600, -0.1308,  0.3478]],
│           
│                   [[ 1.1330, -1.7494, -0.0496],
│                    [-1.0600, -0.1308,  0.3478]],
│           
│                   [[ 1.1330, -1.7494, -0.0496],
│                    [-1.0600, -0.1308,  0.3478]]])
└── 'x' --> <FastTreeValue 0x7f13e59a8d00>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.6 µs ± 44.6 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f13f5bdd520>
├── 'a' --> tensor([[ 1.1330, -1.7494, -0.0496],
│                   [-1.0600, -0.1308,  0.3478],
│                   [ 1.1330, -1.7494, -0.0496],
│                   [-1.0600, -0.1308,  0.3478],
│                   [ 1.1330, -1.7494, -0.0496],
│                   [-1.0600, -0.1308,  0.3478],
│                   [ 1.1330, -1.7494, -0.0496],
│                   [-1.0600, -0.1308,  0.3478],
│                   [ 1.1330, -1.7494, -0.0496],
│                   [-1.0600, -0.1308,  0.3478],
│                   [ 1.1330, -1.7494, -0.0496],
│                   [-1.0600, -0.1308,  0.3478],
│                   [ 1.1330, -1.7494, -0.0496],
│                   [-1.0600, -0.1308,  0.3478],
│                   [ 1.1330, -1.7494, -0.0496],
│                   [-1.0600, -0.1308,  0.3478]])
└── 'x' --> <FastTreeValue 0x7f14bd87c040>
    └── 'c' --> tensor([[ 0.5837,  0.9151,  0.2042,  1.4552],
                        [ 0.3049, -0.9904,  1.6513, -0.3715],
                 

In [23]:
%timeit t_cat(trees)

30.5 µs ± 86.8 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

68.7 µs ± 89.2 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[ 0.5837,  0.9151,  0.2042,  1.4552],
                       [ 0.3049, -0.9904,  1.6513, -0.3715],
                       [ 1.5892,  0.4484, -1.1044,  0.1686]],
              
                      [[ 0.5837,  0.9151,  0.2042,  1.4552],
                       [ 0.3049, -0.9904,  1.6513, -0.3715],
                       [ 1.5892,  0.4484, -1.1044,  0.1686]],
              
                      [[ 0.5837,  0.9151,  0.2042,  1.4552],
                       [ 0.3049, -0.9904,  1.6513, -0.3715],
                       [ 1.5892,  0.4484, -1.1044,  0.1686]],
              
                      [[ 0.5837,  0.9151,  0.2042,  1.4552],
                       [ 0.3049, -0.9904,  1.6513, -0.3715],
                       [ 1.5892,  0.4484, -1.1044,  0.1686]],
              
                      [[ 0.5837,  0.9151,  0.2042,  1.4552],
                       [ 0.3049, -0.9904,  1.6513, -0.3715],
                       [ 1.5892,  0.4484, -1.1044,  0.1686]],

In [26]:
%timeit Batch.stack(batches)

79.4 µs ± 288 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[ 0.5837,  0.9151,  0.2042,  1.4552],
                      [ 0.3049, -0.9904,  1.6513, -0.3715],
                      [ 1.5892,  0.4484, -1.1044,  0.1686],
                      [ 0.5837,  0.9151,  0.2042,  1.4552],
                      [ 0.3049, -0.9904,  1.6513, -0.3715],
                      [ 1.5892,  0.4484, -1.1044,  0.1686],
                      [ 0.5837,  0.9151,  0.2042,  1.4552],
                      [ 0.3049, -0.9904,  1.6513, -0.3715],
                      [ 1.5892,  0.4484, -1.1044,  0.1686],
                      [ 0.5837,  0.9151,  0.2042,  1.4552],
                      [ 0.3049, -0.9904,  1.6513, -0.3715],
                      [ 1.5892,  0.4484, -1.1044,  0.1686],
                      [ 0.5837,  0.9151,  0.2042,  1.4552],
                      [ 0.3049, -0.9904,  1.6513, -0.3715],
                      [ 1.5892,  0.4484, -1.1044,  0.1686],
                      [ 0.5837,  0.9151,  0.2042,  1.4552],
                   

In [28]:
%timeit Batch.cat(batches)

146 µs ± 303 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

319 µs ± 11.7 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
